## Chat GPT 4 Build

In [37]:
import requests
from bs4 import BeautifulSoup
import tkinter as tk
from tkinter import StringVar, ttk
from functools import partial
from tkinter import ttk, Text, Scrollbar
from collections import Counter, defaultdict
from functools import partial

### Import Word Lists

In [38]:

def get_frequency_list():
    # Define the file name
    file_name = "five_letter_frequency_list.txt"

    # Initialize an empty dictionary
    word_frequency_dict = {}

    # Read the contents of the file and parse into a dictionary
    try:
        with open(file_name, "r") as file:
            lines = file.readlines()
            for line in lines:
                word, number = line.strip().split('\t')
                word_frequency_dict[word] = int(number)
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")

    return word_frequency_dict


In [39]:
def remove_words(five_letter_words, OldWords):
    """Removes all words from `five_letter_words` that are also in `OldWords`."""
    return [word for word in five_letter_words if word not in OldWords]

def count_letter_frequency(words_list):
    """Counts the frequency of each letter in the given list of words."""
    letter_frequency = Counter("".join(words_list))
    return sorted(letter_frequency.items(), key=lambda x: x[1], reverse=True)

def filter_words_based_on_letters(words_list, top_common_letters, num_letters=5):
    """Filters the words based on the most common letters and extracts unique letters."""
    filtered_words = words_list.copy()
    for letter, _ in top_common_letters[:num_letters]:
        filtered_words = [word for word in filtered_words if letter in word]
    
    unique_letters = set("".join(filtered_words))
    return filtered_words, list(unique_letters)

def most_common_combinations(words_list, starting_letters, top_n=5):
    """Finds the most common two-letter combinations in the given list of words."""
    two_letter_combinations = Counter()
    for word in words_list:
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                if word[i] in starting_letters and word[j] in starting_letters:
                    combo = tuple(sorted([word[i], word[j]]))
                    two_letter_combinations[combo] += 1
    return two_letter_combinations.most_common(top_n)

def filter_words_by_locked_positions(words_list, locked_positions):
    """Filters words that have the locked letters in the locked positions."""
    for position, letter in locked_positions.items():
        words_list = [word for word in words_list if len(word) >= position and word[position-1] == letter]
        ## OK
        print("letter (lock): ", letter)
        print("Words List Update: ", words_list[:5])
    return words_list

def filter_words_by_excluded_positions(words_list, excluded_positions):
    """Filters out words that have the specified letters in the excluded positions."""
    for position, letter in excluded_positions.items():
        words_list = [word for word in words_list if len(word) < position or word[position-1] != letter]
    return words_list

def exclude_words_with_letters(words_list, excluded_letters):
    """Excludes words that contain any of the letters marked as incorrect by the user."""
    for letter in excluded_letters:
        words_list = [word for word in words_list if letter not in word]
        # ## OK
        # print("letter: ", letter)
        # print("Words List Update: ", words_list[:5])
    return words_list

def filter_words_by_wrong_positions(words_list, excluded_letters):
    # print('\nNew function, these letter must be included')
    # print('Wordlist before ', words_list[:10])
    # print('excluded_letters: ',excluded_letters)

    for _, letter in excluded_letters.items():
         words_list =  [word for word in words_list if letter in word]
    print('Wordlist after :', words_list[:10])
    return words_list

def filter_words_based_on_parameters_updated(words_list, letters_included, letters_not_included, locked_positions, excluded_positions):
    """Filters the word list based on the specified parameters."""
    print('Filter_words_based_on_parameters_updated Started')

    # Filter by letters not included
    words_list = exclude_words_with_letters(words_list, letters_not_included)

    # Filter by locked positions
    words_list = filter_words_by_locked_positions(words_list, locked_positions)
   
    # Filter by excluded positions
    words_list = filter_words_by_excluded_positions(words_list, excluded_positions)
    
    # Filter words list must inclue "Wrong Place" words
    words_list = filter_words_by_wrong_positions(words_list, excluded_positions)

    return words_list

def filter_word_frequency_dict(word_frequency_dict, words_list):

    filtered_dict = {}

    for word in words_list:
        if word in word_frequency_dict:
            filtered_dict[word] = word_frequency_dict[word]
    return filtered_dict




## Recomendations for next word, Scoring

In [40]:
def compute_letter_scores(words_list):
    """Computes the frequency scores for each letter in the words list."""
    letter_frequency = Counter("".join(words_list))
    total_letters = sum(letter_frequency.values())
    
    # Normalize the letter scores
    letter_scores = {letter: freq / total_letters for letter, freq in letter_frequency.items()}
    
    return letter_scores

def compute_unique_letter_scores(words_list):
    """Computes and returns a dictionary with words as keys and their unique letter scores as values, sorted by highest score."""
    
    # Count the frequency of each letter in the entire words_list
    letter_frequency = Counter("".join(words_list))
    
    # For each word, sum the scores of its unique letters
    word_scores = {word: sum([letter_frequency[letter] for letter in set(word)]) for word in words_list}
    
    # Sort the dictionary by score in descending order
    sorted_word_scores = dict(sorted(word_scores.items(), key=lambda item: item[1], reverse=True))

    # Print the top 10 words and their scores
    print("\nTop 10 Unique letter score ")
    for i, (word, score) in enumerate(sorted_word_scores.items()):
        if i == 10:
            break
        print(f"{word}: {score}")
    return sorted_word_scores  

def compute_combination_scores(words_list):
    """Computes the scores for two-letter combinations in the words list."""
    two_letter_combinations = most_common_combinations(words_list,unique_letters, top_n=None)
    total_combinations = sum([count for _, count in two_letter_combinations])
    
    # Normalize the combination scores
    combination_scores = {combo: count / total_combinations for combo, count in two_letter_combinations}
    return combination_scores

def top_n_recommended_words_with_scores(words_list, n=20, repeat_multiplier=0.5):
    print('top_n_recommended_words_with_scores STARTED')
    top_common_letters = count_letter_frequency(words_list) ##!!
    """Recommends the top n optimal words based on letter and combination scores and returns the breakdown."""
    letter_scores = compute_letter_scores(words_list)
    combination_scores = compute_combination_scores(words_list)
    unique_letters_score = compute_unique_letter_scores(words_list)

    repeat_multiplier = float(repeat_multiplier_entry.get()) #Sets the multiplyer via the GUI

    word_scores = {}
    word_letter_scores = {}
    word_combination_scores = {}
    for word in words_list:
        # Letter score
        word_letter_score = sum([letter_scores.get(letter, 0) for letter in word])
        word_letter_scores[word] = round(word_letter_score, 2)
        
        # Combination score
        word_combination_score = 0
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                combo = tuple(sorted([word[i], word[j]]))
                word_combination_score += combination_scores.get(combo, 0)
        word_combination_scores[word] = round(word_combination_score, 2)
        
        # Apply the multiplier if the word has repeated letters
        multiplier = repeat_multiplier if len(word) > len(set(word)) else 1.0
                
        # Total score
        word_scores[word] = round((word_letter_score * multiplier) + word_combination_score, 2)
    
    # Get the top n words with the highest scores
    n=20
    sorted_words = sorted(word_scores, key=word_scores.get, reverse=True)[:n]

    return sorted_words, [word_scores[word] for word in sorted_words], [word_letter_scores[word] for word in sorted_words], [word_combination_scores[word] for word in sorted_words], unique_letters_score


## Graphical interphase

In [41]:
# Command functions for buttons

def create_lock_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = True
        letters_states[position]['excluded'] = False
        letters_states[position]['wrong_place'] = False

        lock_btn.config(bg="green")
        exclude_btn.config(bg="SystemButtonFace")
        wrong_place_btn.config(bg="SystemButtonFace")
    return command

def create_exclude_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = False
        letters_states[position]['excluded'] = True
        letters_states[position]['wrong_place'] = False

        lock_btn.config(bg="SystemButtonFace")
        exclude_btn.config(bg="red")
        wrong_place_btn.config(bg="SystemButtonFace")
    return command

def create_wrong_place_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = False
        letters_states[position]['excluded'] = False
        letters_states[position]['wrong_place'] = True

        lock_btn.config(bg="SystemButtonFace")
        exclude_btn.config(bg="SystemButtonFace")
        wrong_place_btn.config(bg="yellow")
    return command

def wrong_place(button, letter, position):
    global letters_not_included, excluded_positions  # Declare the variables as global
    excluded_positions[position] = letter
    letters_not_included.append(letter)
    #button.config(state=tk.DISABLED)
    button.config(bg='yellow')

    # Add the print statements here
    print("After marking a letter as in the wrong place:")
    print("Letters not included:", letters_not_included)
    print("Locked positions:", locked_positions)
    print("Excluded positions:", excluded_positions)

def display_letters():

    global submitted_word  # Declare it as global
    submitted_word = word_entry.get().upper()  # Retrieve the word from the entry widget
    solve_wordle()
    # Initialize the local state for each letter
    letters_states = [{} for _ in submitted_word]

    for widget in root.winfo_children():
        widget.grid_forget()  # Clear the previous widgets

    # Initialize the local state for each letter
    letters_states = {i: {'locked': False, 'excluded': False, 'wrong_place': False} for i in range(5)}
    
    def solve_wordle_local():
        print('Solve Word Button pressed')
        global locked_positions, excluded_positions, letters_not_included, current_word_list ##!!

        locked_positions = {}
        excluded_positions = {}
        letters_not_included = []

        for position, state in letters_states.items():
            if state['locked']:
                locked_positions[position+1] = submitted_word[position]
            elif state['excluded']:
                letters_not_included.append(submitted_word[position])
            elif state['wrong_place']:
                excluded_positions[position+1] = submitted_word[position]

        temp = solve_wordle() ##!!
        ##print("temp: ", temp) ##!!

        current_word_list = filter_words_based_on_parameters_updated(
        current_word_list, 
        letters_included, 
        letters_not_included, 
        locked_positions, 
        excluded_positions)

        return temp ##!!

    solve_btn.config(command=solve_wordle_local)


    # Create all buttons first
    lock_buttons = []
    exclude_buttons = []
    wrong_place_buttons = []

    for i, letter in enumerate(submitted_word):
        ttk.Label(root, text=letter, width=5).grid(row=1, column=i+1)

        lock_button = tk.Button(root, text="Lock")
        lock_button.grid(row=2, column=i+1)
        lock_buttons.append(lock_button)

        exclude_button = tk.Button(root, text="Exclude")
        exclude_button.grid(row=3, column=i+1)
        exclude_buttons.append(exclude_button)

        wrong_place_button = tk.Button(root, text="Wrong Place")
        wrong_place_button.grid(row=4, column=i+1)
        wrong_place_buttons.append(wrong_place_button)

    # Now configure their commands
    for i, letter in enumerate(submitted_word):
        lock_buttons[i].config(command=create_lock_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))
        exclude_buttons[i].config(command=create_exclude_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))
        wrong_place_buttons[i].config(command=create_wrong_place_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))


    submit_btn.grid(row=0, column=2, padx=10, pady=10)
    print('Submit Word Button pressed')
    solve_btn.grid(row=0, column=3, columnspan=5, pady=10)

    output_text.grid(row=7, column=0, columnspan=6, padx=10, pady=10)
    scrollbar.grid(row=7, column=6, sticky='ns')
    word_entry.grid(row=0, column=1, padx=10, pady=10)

def solve_wordle():
    global word_frequency_dict

    print('solve_wordle Started')
    word = submitted_word
    global letters_included, letters_not_included, locked_positions, excluded_positions, filtered_words_updated
    
    # Get the word from the entry
    word = submitted_word.upper()
    
    # Filter the words
    filtered_words_updated = filter_words_based_on_parameters_updated(current_word_list, letters_included, letters_not_included, locked_positions, excluded_positions)

    # Recommend the next word
    top_words, top_total_scores, top_letter_scores, top_combination_scores, unique_letter_scores = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters)
    print("")

    # Update word_frequency_dict
    word_frequency_dict = filter_word_frequency_dict(word_frequency_dict, words_list)
    
    # Get the first 10 items as a list of tuples
    first_10_items = list(word_frequency_dict.items())[:10]

    # Print the first 10 instances
    print('word_frequency_dict')
    for word, value in first_10_items:
        print(f"{word}: {value}")

    # Prepare the output to be displayed
    header = "       Tot  Letter Combo"
    output_content = [header]
    word_rows  = [f"{word} {total_score}, {letter_score}, {combination_scores}" for word, total_score, letter_score, combination_scores in zip(top_words, top_total_scores, top_letter_scores, top_combination_scores)]
    print('Recomended words')
    print(word_rows)


    unique_letter_scores_output = [f"{word} {score}" for word, score in unique_letter_scores.items()]
    print('unique_letter_scores', unique_letter_scores_output)

    # Append the word rows to the output content
    output_content.extend(word_rows)
    output_content.extend(unique_letter_scores_output)

    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, '\n'.join(output_content))

    return filtered_words_updated
    


In [42]:
# Load all possible words from git 

five_letter_words = fetch_gist_content("https://gist.github.com/dracos/dd0668f281e685bad51479e5acaadb93")
print('Five_letter_words collected')
oldwords = get_old_words()
print('Old Words Collected')
current_word_list = remove_words(five_letter_words, oldwords)
print("Current_word_list set, old words collected")
word_frequency_dict = get_frequency_list()
print("Get word_frequency_dict completed")

top_common_letters = count_letter_frequency(current_word_list)
filtered_words, unique_letters = filter_words_based_on_letters(current_word_list, top_common_letters)

# Initialize the main window
root = tk.Tk()
root.title("Wordle Solver")

# Variables to store user's choices
letters_included = []
letters_not_included = []
locked_positions = {}
excluded_positions = {}
submitted_word = "" 
letter_states = {}  # {position: "locked"/"excluded"/"wrong_place"}

# Button for Submit Word
submit_btn = tk.Button(root, text="Submit Word", command=display_letters)
submit_btn.grid(row=0, column=2, padx=10, pady=10)
print('Submit Word Button pressed')

# Button to Solve Wordle
solve_btn = tk.Button(root, text="Solve Wordle", command=solve_wordle)
solve_btn.grid(row=0, column=3, columnspan=5, pady=10)
print('Solve Word Button pressed')

# Text widget to display multiple lines of output
output_text = Text(root, width=50, height=22)
output_text.grid(row=7, column=0, columnspan=6, padx=10, pady=10)
scrollbar = Scrollbar(root, command=output_text.yview)
scrollbar.grid(row=7, column=6, sticky='nsew')
output_text.config(yscrollcommand=scrollbar.set)

# Entry for the next word
word_entry = ttk.Entry(root, width=15)
word_entry.grid(row=0, column=1, padx=10, pady=10)
word_entry.insert(0, "AEROS") #Suggested starting word

# Create a label for clarity
repeat_multiplier_label = ttk.Label(root, text="Repeat Multiplier:")
repeat_multiplier_label.grid(row=6, column=3, padx=10, pady=10, sticky='e')  # Adjust the row and column as needed

# The actual entry
repeat_multiplier_entry = ttk.Entry(root, width=5)
repeat_multiplier_entry.grid(row=6, column=4, padx=10, pady=10)
repeat_multiplier_entry.insert(0, "0.5")  # Default value

root.mainloop()


Five_letter_words collected
Old Words Collected
Current_word_list set, old words collected
Get word_frequency_dict completed
Submit Word Button pressed
Solve Word Button pressed
solve_wordle Started
Filter_words_based_on_parameters_updated Started
Wordlist after : ['AAHED', 'AALII', 'AAPAS', 'AARGH', 'AARTI', 'ABACA', 'ABACI', 'ABACS', 'ABAFT', 'ABAHT']
top_n_recommended_words_with_scores STARTED

Top 10 Unique letter score 
AEROS: 30108
AROSE: 30108
SOARE: 30108
AESIR: 29347
ARISE: 29347
RAISE: 29347
REAIS: 29347
SERAI: 29347
SERIA: 29347
ALOES: 29270



Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\JonasCarlsson\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\JonasCarlsson\AppData\Local\Temp\ipykernel_20044\2348566909.py", line 53, in display_letters
    solve_wordle()
  File "C:\Users\JonasCarlsson\AppData\Local\Temp\ipykernel_20044\2348566909.py", line 145, in solve_wordle
    word_frequency_dict = filter_word_frequency_dict(word_frequency_dict, words_list)
                                                                          ^^^^^^^^^^
NameError: name 'words_list' is not defined
